In [1]:
# 라이브러리 준비
import tensorflow as tf
import pandas as pd

In [ ]:
# 데이터 준비
(independent, dependent), _ = tf.keras.datasets.mnist.load_data()
independent = independent.reshape(60000, 28, 28, 1)
dependent = pd.get_dummies(dependent)
print(independent.shape, dependent.shape)

In [ ]:
# 모델 생성
X = tf.keras.layers.Input(shape=[28, 28, 1])
H = tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X)
H = tf.keras.layers.Conv2D(6, kernel_size=5, activation='swish')(H)
H = tf.keras.layers.Flatten()(H)
H = tf.keras.layers.Dense(84, activation='swish')(H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [ ]:
# 모델 학습
model.fit(independent, dependent, epochs=10)

In [ ]:
# 모델 이용
pred = model.predict(independent[0:5])
pd.DataFrame(pred).round(2)

In [ ]:
# 정답 확인
dependent[0:5]

In [ ]:
# 모델 확인
model.summary()